### https://github.com/seongsilheo/stress_classification_with_PPG/tree/master

# Peak_detection

Method 1 ) local minima and maxima

In [1]:
#주어진 데이터셋에서 평균값을 기준으로 피크를 탐지하고, 탐지된 피크들 사이 간격을 검사하여 유효한 피크만을 반환
def threshold_peakdetection(dataset, fs):
    
    #print("dataset: ",dataset)
    window = []
    peaklist = []
    ybeat = []
    listpos = 0
    mean = np.average(dataset)
    TH_elapsed = np.ceil(0.36 * fs)
    npeaks = 0
    peakarray = []
    
    localaverage = np.average(dataset)
    for datapoint in dataset:

        if (datapoint < localaverage) and (len(window) < 1):
            listpos += 1
        elif (datapoint >= localaverage):
            window.append(datapoint)
            listpos += 1
        else:
            maximum = max(window)
            beatposition = listpos - len(window) + (window.index(max(window)))
            peaklist.append(beatposition)
            window = []
            listpos += 1

            
    ## Ignore if the previous peak was within 360 ms interval becasuse it is T-wave
    for val in peaklist:
        if npeaks > 0:
            prev_peak = peaklist[npeaks - 1]
            elapsed = val - prev_peak
            if elapsed > TH_elapsed:
                peakarray.append(val)
        else:
            peakarray.append(val)
            
        npeaks += 1    


    return peaklist

Method 2 ) first derivative with adaptive threshold

Reference:

1. Li, Bing Nan, Ming Chui Dong, and Mang I. Vai. "On an automatic delineator for 
arterial blood pressure waveforms." Biomedical Signal Processing and Control 5.1 (2010): 76-81.

2. Elgendi, Mohamed, et al. "Systolic peak detection in acceleration photoplethysmograms 
measured from emergency responders in tropical conditions." PLoS One 8.10 (2013).


In [2]:
#입력 데이터를 5초 단위로 나눠줌
def seperate_division(data,fs):
    divisionSet = []
    for divisionUnit in range(0,len(data)-1,5*fs):  # index of groups (per 5sec) 
        eachDivision = data[divisionUnit: (divisionUnit+1) * 5 * fs]
        divisionSet.append(eachDivision)
    return divisionSet

#각 블록에서 일차 도함수를 계산하여 피크를 탐지, 이 과정에서 각 블록의 첫 2초 동안의 신호 평균을 임계값으로 사용
#신호가 임계값 이상일 때만 피크로 간주, 피크간 최소 간격이 300ms 이상이어야 함
def first_derivative_with_adaptive_ths(data, fs):
    
    peak = []
    divisionSet = seperate_division(data, fs)
    selectiveWindow = 2 * fs
    block_size = 5 *  fs
    bef_idx = -300
    
    for divInd in range(len(divisionSet)):
        block = divisionSet[divInd]
        ths = np.mean(block[:selectiveWindow]) # ths: 2 seconds mean in block
        
        firstDeriv = block[1:] - block[:-1]
        for i in range(1,len(firstDeriv)):
            if  firstDeriv[i] <= 0 and firstDeriv[i-1] > 0:
                if block[i] > ths:
                    idx = block_size*divInd + i
                    if idx - bef_idx > (300*fs/1000):
                        peak.append(idx)
                        bef_idx = idx
                                                
    return peak

Method 3: Slope sum function with an adaptive threshold

Reference
1. Jang, Dae-Geun, et al. "A robust method for pulse peak determination 
in a digital volume pulse waveform with a wandering baseline." 
IEEE transactions on biomedical circuits and systems 8.5 (2014): 729-737.

2. Jang, Dae-Geun, et al. "A real-time pulse peak detection algorithm for 
the photoplethysmogram." International Journal of Electronics and Electrical Engineering 2.1 (2014): 45-49.

In [3]:
#현재값이 이전 값보다 크가 다음 값보다 크거나 같으면 피크
def determine_peak_or_not(prevAmp, curAmp, nextAmp):
    if prevAmp < curAmp and curAmp >= nextAmp:
        return True
    else:
        return False
    
# 피크의 시작점과 끝점을 계산    
def onoff_set(peak, sig):     # move peak from dy signal to original signal   
    onoffset = []
    for p in peak:
        for i in range(p, 0,-1):
            if sig[i] == 0:
                onset = i
                break
        for j in range(p, len(sig)):
            if sig[j] == 0:
                offset = j
                break
        if onset < offset:
            onoffset.append([onset,offset])
    return onoffset
    

#신호의 변화율 (미분 값)을 사용하여 피크를 탐지하고 , 이를 기반으로 정확한 피크 위치를 반환하는 전체 프로세스(위 두함수 사용)
def slope_sum_function(data,fs):
    dy = [0]
    
    dy.extend(np.diff(data))
    #dy[dy < 0 ] = 0
    
    w = fs // 8
    dy_ = [0] * w
    for i in range(len(data)-w):
        sum_ = np.sum(dy[i:i+w])
        if sum_ > 0:
            dy_.append(sum_)
        else:
            dy_.append(0)
    
    init_ths = 0.6 * np.max(dy[:3*fs])
    ths = init_ths
    recent_5_peakAmp = []
    peak_ind = []
    bef_idx = -300
    
    for idx in range(1,len(dy_)-1):
        prevAmp = dy_[idx-1]
        curAmp = dy_[idx]
        nextAmp = dy_[idx+1]
        if determine_peak_or_not(prevAmp, curAmp, nextAmp) == True:
            if (idx - bef_idx) > (300 * fs /1000):  # Ignore if the previous peak was within 300 ms interval
                if len(recent_5_peakAmp) < 100:  
                    if curAmp > ths:
                        peak_ind.append(idx)
                        bef_idx = idx
                        recent_5_peakAmp.append(curAmp)
                elif len(recent_5_peakAmp) == 100:
                    ths = 0.7*np.median(recent_5_peakAmp)
                    if curAmp > ths:
                        peak_ind.append(idx)
                        bef_idx = idx
                        recent_5_peakAmp.pop(0)
                        recent_5_peakAmp.append(curAmp)
                        
    onoffset = onoff_set(peak_ind, dy_)
    corrected_peak_ind = []
    for onoff in onoffset:
        segment = data[onoff[0]:onoff[1]]
        corrected_peak_ind.append(np.argmax(segment) + onoff[0])
                    
    return corrected_peak_ind

Method 4

Event-Related Moving Averages with Dynamic Threshold

Method 4: 사건 관련 이동 평균과 동적 임계값

Reference

1. Elgendi, Mohamed, et al. "Systolic peak detection in acceleration photoplethysmograms 
measured from emergency responders in tropical conditions." PLoS One 8.10 (2013).

2. https://github.com/neuropsychology/NeuroKit/blob/8a2148fe477f20328d18b6da7bbb1c8438e60f18/neurokit2/signal/signal_formatpeaks.py


In [4]:
# 신호의 이동 평균을 계산

# 윈도우(필터)의 종류는 boxcar(평균필터)
def moving_average(signal, kernel='boxcar', size=5):    
    size = int(size)
    window = scipy.signal.get_window(kernel, size)
    w = window / window.sum()
    
    # Extend signal edges to avoid boundary effects
    x = np.concatenate((signal[0] * np.ones(size), signal, signal[-1] * np.ones(size)))
    
    # Compute moving average
    smoothed = np.convolve(w, x, mode='same')
    smoothed = smoothed[size:-size]
    return smoothed


# 주어진 신호에서 피크 탐지
def moving_averages_with_dynamic_ths(signals,sampling_rate=64, peakwindow=.111, 
                                     beatwindow=.667, beatoffset=.02, mindelay=.3,show=False):
    if show:
        fig, (ax0, ax1) = plt.subplots(nrow=2, ncols=1, sharex=True)
        ax0.plot(data, label='filtered')
    
    signal = signals.copy()
    # ignore the samples with n
    signal[signal < 0] = 0
    sqrd = signal**2
    
    # Compute the thresholds for peak detection. Call with show=True in order
    # to visualize thresholds.
    ma_peak_kernel = int(np.rint(peakwindow * sampling_rate))
    ma_peak = moving_average(sqrd, size=ma_peak_kernel)
    
    ma_beat_kernel = int(np.rint(beatwindow * sampling_rate))
    ma_beat = moving_average(sqrd, size=ma_beat_kernel)

    
    thr1 = ma_beat + beatoffset * np.mean(sqrd)    # threshold 1

    if show:
        ax1.plot(sqrd, label="squared")
        ax1.plot(thr1, label="threshold")
        ax1.legend(loc="upper right")

    # Identify start and end of PPG waves.
    waves = ma_peak > thr1
    
    beg_waves = np.where(np.logical_and(np.logical_not(waves[0:-1]),
                                        waves[1:]))[0]
    end_waves = np.where(np.logical_and(waves[0:-1],
                                        np.logical_not(waves[1:])))[0]
    # Throw out wave-ends that precede first wave-start.
    end_waves = end_waves[end_waves > beg_waves[0]]

    # Identify systolic peaks within waves (ignore waves that are too short).
    num_waves = min(beg_waves.size, end_waves.size)
    min_len = int(np.rint(peakwindow * sampling_rate))    # threshold 2
    min_delay = int(np.rint(mindelay * sampling_rate))
    peaks = [0]

    for i in range(num_waves):

        beg = beg_waves[i]
        end = end_waves[i]
        len_wave = end - beg

        if len_wave < min_len: # threshold 2
            continue

        # Visualize wave span.
        if show:
            ax1.axvspan(beg, end, facecolor="m", alpha=0.5)

        # Find local maxima and their prominence within wave span.
        data = signal[beg:end]
        locmax, props = scipy.signal.find_peaks(data, prominence=(None, None))

        if locmax.size > 0:
            # Identify most prominent local maximum.
            peak = beg + locmax[np.argmax(props["prominences"])]
            # Enforce minimum delay between peaks(300ms)
            if peak - peaks[-1] > min_delay:
                peaks.append(peak)

    peaks.pop(0)

    if show:
        ax0.scatter(peaks, signal[peaks], c="r")

    peaks = np.asarray(peaks).astype(int)
    return peaks

In [5]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def lmm_peakdetection(data,fs):
    
    peak_final = []
    peaks, _ = find_peaks(data,height=0)
    
    for peak in peaks:
        if data[peak] > 0:
            peak_final.append(peak)
        
    return peak_final

In [6]:
def ensemble_peak(preprocessed_data, fs, ensemble_ths=4):
    
    peak1 = threshold_peakdetection(preprocessed_data,fs)
    peak2 = slope_sum_function(preprocessed_data, fs)
    peak3 = first_derivative_with_adaptive_ths(preprocessed_data, fs)
    peak4 = moving_averages_with_dynamic_ths(preprocessed_data)
    peak5 = lmm_peakdetection(preprocessed_data,fs)
    
    peak_dic = dict()

    for key in peak1:
        peak_dic[key] = 1

    for key in peak2:
        if key in peak_dic.keys():
            peak_dic[key] += 1
        else:
            peak_dic[key] = 1
    
    for key in peak3:
        if key in peak_dic.keys():
            peak_dic[key] += 1
        else:
            peak_dic[key] = 1
        
    for key in peak4:
        if key in peak_dic.keys():
            peak_dic[key] += 1
        else:
            peak_dic[key] = 1
        
    for key in peak5:
        if key in peak_dic.keys():
            peak_dic[key] += 1
        else:
            peak_dic[key] = 1
        
    peak_dic = dict(sorted(peak_dic.items()))

    count = 0
    cnt = 0
    bef_key = 0
    margin = 1

    new_peak_dic = dict()

    for key in peak_dic.keys():
        if cnt == 0:
            new_peak_dic[key] = peak_dic[key]
        else:
            if bef_key + margin >= key:  # 마진 1안에 다음 피크가 존재하면
                if peak_dic[bef_key] > peak_dic[key]: # 이전 피크 기준으로 개수 카운트
                    new_peak_dic[bef_key] += peak_dic[key]
                else:
                    #print("new peak dic: ",new_peak_dic)
                    new_peak_dic[key] = peak_dic[key] + peak_dic[bef_key] # 현재 피크 기준으로 개수 카운트
                    del(new_peak_dic[bef_key])
                    bef_key = key
            else:
                new_peak_dic[key] = peak_dic[key]
                bef_key = key
        cnt += 1
    
    ensemble_dic = dict()
    for (key, value) in new_peak_dic.items():
        if value >= ensemble_ths:
            ensemble_dic[key] = value
            
    final_peak = list(ensemble_dic.keys())
    
    return final_peak

# noise_reduction

+
filter

In [7]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs  
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpassfilter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y



def movingaverage(data, periods=4):
    result = []
    data_set = np.asarray(data)
    weights = np.ones(periods) / periods
    result = np.convolve(data_set, weights, mode='valid')
    return result

-

In [8]:
def detrend_signals(signals):
    detrended_signals = []
    X = [k for k in range(0, len(signals))]
    X = np.reshape(X, (len(X), 1)) ## Reshapes the array into necessary format
    model = LinearRegression() ## Defines the model
    model.fit(X, signals) ## Fits signals to the model
    trend = model.predict(X) ## Predicts the model's trend
    detrend = [signals[k] - trend[k] for k in range(len(signals))] ## Removes trend from signal
    detrended_signals.append(detrend)
    
    return detrended_signals[0]

Method 1 ) local minima and maxima

In [9]:
def threshold_peakdetection(dataset, fs):
    
    #print("dataset: ",dataset)
    window = []
    peaklist = []
    ybeat = []
    listpos = 0
    mean = np.average(dataset)
    TH_elapsed = np.ceil(0.36 * fs)
    npeaks = 0
    peakarray = []
    
    localaverage = np.average(dataset)
    for datapoint in dataset:

        if (datapoint < localaverage) and (len(window) < 1):
            listpos += 1
        elif (datapoint >= localaverage):
            window.append(datapoint)
            listpos += 1
        else:
            maximum = max(window)
            beatposition = listpos - len(window) + (window.index(max(window)))
            peaklist.append(beatposition)
            window = []
            listpos += 1

            
    ## Ignore if the previous peak was within 360 ms interval becasuse it is T-wave
    for val in peaklist:
        if npeaks > 0:
            prev_peak = peaklist[npeaks - 1]
            elapsed = val - prev_peak
            if elapsed > TH_elapsed:
                peakarray.append(val)
        else:
            peakarray.append(val)
            
        npeaks += 1    


    return peaklist

In [10]:
def RR_interval(peaklist,fs):
    RR_list = []
    cnt = 0
    while (cnt < (len(peaklist)-1)):
        RR_interval = (peaklist[cnt+1] - peaklist[cnt]) # Calculate distance between beats in # of samples
        ms_dist = ((RR_interval / fs) * 1000.0)  # Convert sample distances to ms distances (fs로 나눠서 1초단위로 거리표현 -> 1ms단위로 change) 
        RR_list.append(ms_dist)
        cnt += 1
        
    return RR_list

In [11]:
def calc_heartrate(RR_list):
    HR = []
    heartrate_array=[]
    window_size = 10

    for val in RR_list:
        if val > 400 and val < 1500:
            heart_rate = 60000.0 / val #60000 ms /1 minute. time per beat(한번 beat하는데 걸리는 시간)
            
        # if RR-interval < .1905 seconds, heart-rate > highest recorded value, 315 BPM. Probably an error!
        elif (val > 0 and val < 400) or val > 1500:
            if len(HR) > 0:
                # ... and use the mean heart-rate from the data so far:
                heart_rate = np.mean(HR[-window_size:])

            else:
                heart_rate = 60.0
        else:
            # Get around divide by 0 error
            heart_rate = 0.0

        HR.append(heart_rate)

    return HR

1. Frequency view

Reference:
Sadhukhan, Deboleena, Saurabh Pal, and Madhuchhanda Mitra. 
"PPG Noise Reduction based on Adaptive Frequency Suppression using Discrete Fourier Transform 
for Portable Home Monitoring Applications." 
2018 15th IEEE India Council International Conference (INDICON). IEEE, 2018.

In [12]:
def FFT(block,fs):
    fourierTransform = np.fft.fft(block)/len(block)  # divide by len(block) to normalize
    fourierTransform = fourierTransform[range(int(len(block)/2))] # single side frequency / symmetric

    tpCount = len(block)
    values = np.arange(int(tpCount)/2)

    timePeriod = tpCount / fs
    frequencies = values/timePeriod # frequency components

    '''
    plt.figure(figsize=(40,20))
    plt.plot(frequencies, abs(fourierTransform)) 

    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.xlabel("frequency(Hz)",fontsize=50)
    plt.ylabel("FFT magnitude(mV)",fontsize=50)
    '''
    
    return frequencies, fourierTransform, timePeriod

In [13]:
def get_cutoff(block,fs):
    
    block = np.array([item.real for item in block])
    peak = threshold_peakdetection(block,fs)
    hr_mean = np.mean(calc_heartrate(RR_interval(peak,fs)))
    #low_cutoff = np.round(hr_mean / 60 - 0.2, 1)
    low_cutoff = np.round(hr_mean / 60 - 0.6, 1) # 0.6
    
    
    frequencies, fourierTransform, timePeriod = FFT(block,fs)
    ths = max(abs(fourierTransform)) * 0.10
    
    for i in range(int(5*timePeriod),0, -1):  # check from 5th harmonic
        if abs(fourierTransform[i]) > ths:
            high_cutoff = np.round(i/timePeriod, 1) 
            break
    
    print('low cutoff: ', low_cutoff, 'high_cutoff: ', high_cutoff)
    
    #low_loc = np.where(frequencies == low_cutoff)[0][0]
    #high_loc = np.where(frequencies == high_cutoff)[0][0]
            
    return [low_cutoff, high_cutoff]


In [14]:
def compute_and_reconstruction_dft(data, fs, sec, overlap, cutoff):
    concatenated_sig = []
    
    for i in range(0, len(data), fs*sec-overlap):
        seg_data = data[i:i+fs*sec]
        sig_fft = fftpack.fft(seg_data)
    
        
        #The corresponding frequencies
        sample_freq = (fftpack.fftfreq(len(seg_data)) * fs)

        new_freq_fft = sig_fft.copy()
        new_freq_fft[np.abs(sample_freq) < cutoff[0]] = 0
        new_freq_fft[np.abs(sample_freq) > cutoff[1]] = 0
    
        filtered_sig = fftpack.ifft(new_freq_fft)
        
        # 2% overlapping
        if i == 0:
            concatenated_sig = np.hstack([concatenated_sig, filtered_sig[:fs*sec - overlap//2]])
        elif i == len(data)-1:
            concatenated_sig = np.hstack(concatenated_sig, filtered_sig[overlap//2:])
        else:
            concatenated_sig = np.hstack([concatenated_sig, filtered_sig[overlap//2:fs*sec - overlap//2]])
        
    return concatenated_sig

+

In [15]:
from sklearn.preprocessing import MinMaxScaler

def ppg_normalization(data, min_range, max_range):

    scaler = MinMaxScaler(feature_range=(min_range,max_range))
    scaler.fit(data)
    norm_data = scaler.transform(data)
    
    return norm_data

Reference:
Hanyu, Shao, and Chen Xiaohui. "Motion artifact detection and reduction in PPG signals based on statistics analysis." 
2017 29th Chinese control and decision conference (CCDC). IEEE, 2017.

cycle: per period

In [16]:
def statistic_threshold(clean_signal, fs, ths):
    stds, kurtosiss, skews, valley = statistic_detection(clean_signal, fs)
    std_ths = np.mean(stds) + ths[0] # paper's threshold : 3.2  
    kurt_ths = np.mean(kurtosiss) + ths[1]  #3.1  
    skews_ths = [np.mean(skews) - ths[2], np.mean(skews) + ths[3]]  # -0.3 and 0.8   -0.5 , 0.9
    
    return std_ths, kurt_ths, skews_ths

In [17]:
def statistic_detection(signal, fs):
    
    valley = pair_valley(valley_detection(signal, fs))
    stds=[]
    kurtosiss=[]
    skews=[]

    for val in valley: # 사이클 한 번동안의 통계적 평균 리스트 저장
        stds.append(np.std(signal[val[0]:val[1]]))
        kurtosiss.append(kurtosis(signal[val[0]:val[1]]))
        skews.append(skew(signal[val[0]:val[1]])) 

    return stds, kurtosiss, skews, valley

In [18]:
def eliminate_noise_in_time(data, fs, ths,cycle=1):
    stds, kurtosiss,skews, valley = statistic_detection(data, fs)
    
    
    #cycle 수만큼 다시 평균내서 리스트 저장
    stds_, kurtosiss_, skews_ = [], [], []
    stds_ = [np.mean(stds[i:i+cycle]) for i in range(0,len(stds)-cycle+1,cycle)]
    kurtosiss_ = [np.mean(kurtosiss[i:i+cycle]) for i in range(0,len(kurtosiss)-cycle+1,cycle)]
    skews_ = [np.mean(skews[i:i+cycle]) for i in range(0,len(skews)-cycle+1,cycle)]    
   
    # extract clean index, 사이클 인덱스       
    eli_std = [stds_.index(x) for x in stds_ if x < ths[0]]
    eli_kurt = [kurtosiss_.index(x) for x in kurtosiss_ if x < ths[1]]
    eli_skew = [skews_.index(x) for x in skews_ if x > ths[2][0] and x < ths[2][1]]

    total_list = eli_std + eli_kurt + eli_skew
    
     # store the number of extracted each index(각 인덱스 extract된 횟수 저장)
    dic = dict()
    for i in total_list:
        if i in dic.keys():
            dic[i] += 1
        else:
            dic[i] = 1
            
    new_list = []
    for key, value in dic.items():
        if value >= 3:
            new_list.append(key)
    new_list.sort()
    
    eliminated_data = []
    index = []
    for x in new_list:
        index.extend([x for x in range(valley[x*cycle][0],valley[x*cycle+cycle-1][1],1)])

    print(len(data), len(index))
    return len(data), len(index), index

+

In [19]:
def valley_detection(dataset, fs):
    window = []
    valleylist = []
    ybeat = []
    listpos = 0
    TH_elapsed = np.ceil(0.36 * fs)
    nvalleys = 0
    valleyarray = []
    
    localaverage = np.average(dataset)
    for datapoint in dataset:

        if (datapoint > localaverage) and (len(window) < 1):
            listpos += 1
        elif (datapoint <= localaverage):
            window.append(datapoint)
            listpos += 1
        else:
            minimum = min(window)
            beatposition = listpos - len(window) + (window.index(min(window)))
            valleylist.append(beatposition)
            window = []
            listpos += 1

    ## Ignore if the previous peak was within 360 ms interval becasuse it is T-wave
    for val in valleylist:
        if nvalleys > 0:
            prev_valley = valleylist[nvalleys - 1]
            elapsed = val - prev_valley
            if elapsed > TH_elapsed:
                valleyarray.append(val)
        else:
            valleyarray.append(val)
            
        nvalleys += 1    

    return valleyarray


In [20]:
def pair_valley(valley):
    pair_valley=[]
    for i in range(len(valley)-1):
        pair_valley.append([valley[i], valley[i+1]])
    return pair_valley

-

In [21]:
def auto_correlation(filtered_3sec):
    
    a = plt.acorr(filtered_3sec, usevlines=True, normed=True, maxlags=(len(filtered_3sec)-1), lw=2)
    valley_indexes = signal.argrelextrema(a[1], np.less)  # find valley index
    plt.scatter(x=valley_indexes[0]-(len(filtered_3sec)-1), y=a[1][valley_indexes[0]], color='red', s=20)
    
    diff = []
    for i in range(len(valley_indexes[0]) -1):
        diff.append(valley_indexes[0][i+1] - valley_indexes[0][i])
    
    return np.average(diff)

+
it doesn't work well when they are noisy signals, a and b gotta be same

In [22]:
'''
def improved_moving_window_method(data, fs, mean_period):
    
    # find first valley
    valley = valley_detection(data[:100],fs)
    final_valley = [valley[0]]
    
    i=0
    while True:
        if i == 0:
            meuw = mean_period
            a = math.ceil(valley[0] + 2/3*meuw)
            b = math.ceil(valley[0] + 4/3*meuw)
        else:
            meuw = final_valley[-1] - final_valley[-2]
            a = math.ceil(final_valley[-1] + 2/3*meuw)
            b = math.ceil(final_valley[-1] + 4/3*meuw)

        final_valley.append(a + np.argmin(data[a:b]))      
        i += 1   
                        
        if b > len(data):
            break
    
    return final_valley
# + {}
'''

'\ndef improved_moving_window_method(data, fs, mean_period):\n    \n    # find first valley\n    valley = valley_detection(data[:100],fs)\n    final_valley = [valley[0]]\n    \n    i=0\n    while True:\n        if i == 0:\n            meuw = mean_period\n            a = math.ceil(valley[0] + 2/3*meuw)\n            b = math.ceil(valley[0] + 4/3*meuw)\n        else:\n            meuw = final_valley[-1] - final_valley[-2]\n            a = math.ceil(final_valley[-1] + 2/3*meuw)\n            b = math.ceil(final_valley[-1] + 4/3*meuw)\n\n        final_valley.append(a + np.argmin(data[a:b]))      \n        i += 1   \n                        \n        if b > len(data):\n            break\n    \n    return final_valley\n# + {}\n'

# feature_extraction

In [23]:
import sys
sys.path.append("stress_classification_with_PPG/preprocessing_tool/") 

In [24]:
def calc_RRI(peaklist, fs):
    RR_list = []
    RR_list_e = []
    cnt = 0
    while (cnt < (len(peaklist)-1)):
        RR_interval = (peaklist[cnt+1] - peaklist[cnt]) #Calculate distance between beats in # of samples
        ms_dist = ((RR_interval / fs) * 1000.0)  #fs로 나눠서 1초단위로 거리표현 -> 1ms단위로 change /  Convert sample distances to ms distances
        cnt += 1
        RR_list.append(ms_dist)
    mean_RR = np.mean(RR_list)

    for ind, rr in enumerate(RR_list):
        if rr >  mean_RR - 300 and rr < mean_RR + 300:
            RR_list_e.append(rr)
            
    RR_diff = []
    RR_sqdiff = []
    cnt = 0
    while (cnt < (len(RR_list_e)-1)):
        RR_diff.append(abs(RR_list_e[cnt] - RR_list_e[cnt+1]))
        RR_sqdiff.append(math.pow(RR_list_e[cnt] - RR_list_e[cnt+1], 2))
        cnt += 1
        
    return RR_list_e, RR_diff, RR_sqdiff


In [25]:
def calc_heartrate(RR_list):
    HR = []
    heartrate_array=[]
    window_size = 10

    for val in RR_list:
        if val > 400 and val < 1500:
            heart_rate = 60000.0 / val #60000 ms (1 minute) / 한번 beat하는데 걸리는 시간
        # if RR-interval < .1905 seconds, heart-rate > highest recorded value, 315 BPM. Probably an error!
        elif (val > 0 and val < 400) or val > 1500:
            if len(HR) > 0:
                # ... and use the mean heart-rate from the data so far:
                heart_rate = np.mean(HR[-window_size:])

            else:
                heart_rate = 60.0
        else:
            # Get around divide by 0 error
            print("err")
            heart_rate = 0.0

        HR.append(heart_rate)

    return HR

In [26]:
def get_window_stats_original(ppg_seg, window_length, label=-1):  # Nan을 제외하고 평균 냄 
    
    fs = 64   
    
    peak = threshold_peakdetection(ppg_seg, fs)
    RR_list, RR_diff, RR_sqdiff = calc_RRI(peak, fs)
    
    # Time
    HR = calc_heartrate(RR_list)
    HR_mean, HR_std = np.mean(HR), np.std(HR)
    SD_mean, SD_std = np.mean(RR_diff) , np.std(RR_diff)
    NN50 = [x for x in RR_diff if x > 50]
    pNN50 = len(NN50) / window_length
    bar_y, bar_x = np.histogram(RR_list)
    TINN = np.max(bar_x) - np.min(bar_x)
    RMSSD = np.sqrt(np.mean(RR_sqdiff))
    
    # Frequency
    rr_x = []
    pointer = 0
    for x in RR_list:
        pointer += x
        rr_x.append(pointer)
    RR_x_new = np.linspace(rr_x[0], rr_x[-1], int(rr_x[-1]))
    
    if len(rr_x) <= 5 or len(RR_list) <= 5:
        print("rr_x or RR_list less than 5")   
    
   
    interpolated_func = UnivariateSpline(rr_x, RR_list, k=3)
    
    datalen = len(RR_x_new)
    frq = np.fft.fftfreq(datalen, d=((1/1000.0)))
    frq = frq[range(int(datalen/2))]
    Y = np.fft.fft(interpolated_func(RR_x_new))/datalen
    Y = Y[range(int(datalen/2))]
    psd = np.power(Y, 2)  # power spectral density

    lf = np.trapz(abs(psd[(frq >= 0.04) & (frq <= 0.15)])) #Slice frequency spectrum where x is between 0.04 and 0.15Hz (LF), and use NumPy's trapezoidal integration function to find the are
    hf = np.trapz(abs(psd[(frq > 0.15) & (frq <= 0.5)])) #Do the same for 0.16-0.5Hz (HF)
    ulf = np.trapz(abs(psd[frq < 0.003]))
    vlf = np.trapz(abs(psd[(frq >= 0.003) & (frq < 0.04)]))
    
    if hf != 0:
        lfhf = lf/hf
    else:
        lfhf = 0
        
    total_power = lf + hf + vlf

    features = {'HR_mean': HR_mean, 'HR_std': HR_std, 'SD_mean': SD_mean, 'SD_std': SD_std, 'pNN50': pNN50, 'TINN': TINN, 'RMSSD': RMSSD,
                'LF': lf, 'HF': hf, 'ULF' : ulf, 'VLF': vlf, 'LFHF': lfhf, 'Total_power': total_power, 'label': label}

    return features

In [27]:
def approximate_entropy(U, m=2, r=3):

    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0) for x_i in x]
        return (N - m + 1.0)**(-1) * sum(np.log(C))

    N = len(U)

    return abs(_phi(m+1) - _phi(m))

In [28]:
def shannon_entropy(signal):
    #signal = list(signal)
    
    data_set = list(set(signal))
    freq_list = []
    for entry in data_set:
        counter = 0.
        for i in signal:
            if i == entry:
                counter += 1
        freq_list.append(float(counter) / len(signal))
        
    ent = 0.0
    for freq in freq_list:
        ent += freq * np.log2(freq)
    
    ent = -ent
    
    return ent

 https://horizon.kias.re.kr/12415

In [29]:
def sample_entropy(sig,ordr,tor):
    # sig: the input signal or series, it should be numpy array with type float
    # ordr: order, the length of template,embedding dimension
    # tor: percent of standard deviation
    
    sig = np.array(sig)
    n = len(sig)
    #tor = np.std(sig)*tor
    
    matchnum = 0.0
    for i in range(n-ordr):
        tmpl = sig[i:i+ordr] # generate samples length ordr
        for j in range (i+1,n-ordr+1): 
            ltmp = sig[j:j+ordr]
            diff = tmpl-ltmp  # measure mean similarity
            if all(diff<tor):
                matchnum+=1
    
    allnum = (n-ordr+1)*(n-ordr)/2
    if matchnum<0.1:
        sen = 1000.0
    else:
        sen = -math.log(matchnum/allnum)
    return sen

In [30]:
def calc_td_hrv(RR_list, RR_diff, RR_sqdiff, window_length): 
    
    # Time
    HR = calc_heartrate(RR_list)
    HR_mean, HR_std = np.mean(HR), np.std(HR)
    meanNN, SDNN, medianNN = np.mean(RR_list), np.std(RR_list), np.median(np.abs(RR_list))
    meanSD, SDSD = np.mean(RR_diff) , np.std(RR_diff)
    RMSSD = np.sqrt(np.mean(RR_sqdiff))
    
    NN20 = [x for x in RR_diff if x > 20]
    NN50 = [x for x in RR_diff if x > 50]
    pNN20 = len(NN20) / window_length
    pNN50 = len(NN50) / window_length
    
    
    bar_y, bar_x = np.histogram(RR_list)
    TINN = np.max(bar_x) - np.min(bar_x)
    
    RMSSD = np.sqrt(np.mean(RR_sqdiff))
    

    features = {'HR_mean': HR_mean, 'HR_std': HR_std, 'meanNN': meanNN, 'SDNN': SDNN, 'medianNN': medianNN,
                'meanSD': meanSD, 'SDSD': SDSD, 'RMSSD': RMSSD, 'pNN20': pNN20, 'pNN50': pNN50, 'TINN': TINN}

    return features

In [31]:
def calc_fd_hrv(RR_list):  
    
    rr_x = []
    pointer = 0
    for x in RR_list:
        pointer += x
        rr_x.append(pointer)
        
    if len(rr_x) <= 3 or len(RR_list) <= 3:
        print("rr_x or RR_list less than 5")   
        return 0
    
    RR_x_new = np.linspace(rr_x[0], rr_x[-1], int(rr_x[-1]))
    
   
    interpolated_func = UnivariateSpline(rr_x, RR_list, k=3)
    
    datalen = len(RR_x_new)
    frq = np.fft.fftfreq(datalen, d=((1/1000.0)))
    frq = frq[range(int(datalen/2))]
    Y = np.fft.fft(interpolated_func(RR_x_new))/datalen
    Y = Y[range(int(datalen/2))]
    psd = np.power(Y, 2)  # power spectral density

    lf = np.trapz(abs(psd[(frq >= 0.04) & (frq <= 0.15)])) #Slice frequency spectrum where x is between 0.04 and 0.15Hz (LF), and use NumPy's trapezoidal integration function to find the are
    hf = np.trapz(abs(psd[(frq > 0.15) & (frq <= 0.5)])) #Do the same for 0.16-0.5Hz (HF)
    ulf = np.trapz(abs(psd[frq < 0.003]))
    vlf = np.trapz(abs(psd[(frq >= 0.003) & (frq < 0.04)]))
    
    if hf != 0:
        lfhf = lf/hf
    else:
        lfhf = 0
        
    total_power = lf + hf + vlf
    lfp = lf / total_power
    hfp = hf / total_power

    features = {'LF': lf, 'HF': hf, 'ULF' : ulf, 'VLF': vlf, 'LFHF': lfhf, 'total_power': total_power, 'lfp': lfp, 'hfp': hfp}
    bef_features = features
    
    return features


In [32]:
def calc_nonli_hrv(RR_list,label): 
    
    diff_RR = np.diff(RR_list)
    sd_heart_period = np.std(diff_RR, ddof=1) ** 2
    SD1 = np.sqrt(sd_heart_period * 0.5)
    SD2 = 2 * sd_heart_period - 0.5 * sd_heart_period
    pA = SD1*SD2
    
    if SD2 != 0:
        pQ = SD1 / SD2
    else:
        print("SD2 is zero")
        pQ = 0
    
    ApEn = approximate_entropy(RR_list,2,3)  
    shanEn = shannon_entropy(RR_list)
    #sampEn = nolds.sampen(RR_list,emb_dim=2)
    D2 = nolds.corr_dim(RR_list, emb_dim=2)
    #dfa1 = nolds.dfa(RR_list, range(4,17))
    # dfa2 = nolds.dfa(RR_list, range(16,min(len(RR_list)-1, 66)))
    #dimension, delay, threshold, norm, minimum_diagonal_line_length = 3, 2, 0.7, "manhattan", 2
    #rec_mat = recurrence_matrix(RR_list, dimension, delay, threshold, norm)
    #REC, RPImean, RPImax, RPadet = recurrence_quantification_analysis(rec_mat, minimum_diagonal_line_length)
    # recurrence_rate, average_diagonal_line_length, longest_diagonal_line_length, determinism

    features = {'SD1': SD1, 'SD2': SD2, 'pA': pA, 'pQ': pQ, 'ApEn' : ApEn, 'shanEn': shanEn, 'D2': D2, 
                'label': label}
    # 'dfa1': dfa1, 'dfa2': dfa2, 'REC': REC, 'RPImean': RPImean, 'RPImax': RPImax, 'RPadet': RPadet,
    return features

In [33]:
def get_window_stats_27_features(ppg_seg, window_length, label, ensemble, ma_usage):  
    
    fs = 64  
    
    if ma_usage:
        fwd = moving_average(ppg_seg, size=3)
        bwd = moving_average(ppg_seg[::-1], size=3)
        ppg_seg = np.mean(np.vstack((fwd,bwd[::-1])), axis=0)
    ppg_seg = np.array([item.real for item in ppg_seg])
    
    #peak = threshold_peakdetection(ppg_seg, fs)
    #peak = first_derivative_with_adaptive_ths(ppg_seg, fs)
    #peak = slope_sum_function(ppg_seg, fs)
    #peak = moving_averages_with_dynamic_ths(ppg_seg)
    peak = lmm_peakdetection(ppg_seg,fs)

        
    if ensemble:
        ensemble_ths = 3
        #print("one algorithm peak length: ", len(peak))
        peak = ensemble_peak(ppg_seg, fs, ensemble_ths)
        #print("after ensemble peak length: ", len(peak))
        
        if(len(peak) < 100):
            print("skip")
            return []

        
    RR_list, RR_diff, RR_sqdiff = calc_RRI(peak, fs)
    #print(RR_list)
    
    if len(RR_list) <= 3:
        return []
    
    td_features = calc_td_hrv(RR_list, RR_diff, RR_sqdiff, window_length)
    fd_features = calc_fd_hrv(RR_list)
    
    if fd_features == 0:
        return []
    nonli_features = calc_nonli_hrv(RR_list,label)
    
    total_features = {**td_features, **fd_features, **nonli_features}
    
    
    return total_features

# read_data_new_tri

In [34]:
!pip install nolds

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 7.3 MB/s eta 0:00:00


In [35]:
import os
import pickle
import math
import numpy as np
import pandas as pd
import scipy
import nolds

from scipy.signal import butter, filtfilt


from scipy.interpolate import UnivariateSpline
from scipy import stats

from scipy.interpolate import UnivariateSpline

In [36]:
import math
from scipy import signal
from scipy import fft, ifft
from scipy import fftpack
from scipy.stats import kurtosis, skew
from scipy.signal import butter, lfilter
from scipy import stats
from sklearn.linear_model import LinearRegression



import numpy as np

import matplotlib.pyplot as plt

In [37]:
# # +
WINDOW_IN_SECONDS = 120  # 120 / 180 / 300

NOISE = ['bp_time_ens']
main_path='WESAD/WESAD/'

# +
# E4 (wrist) Sampling Frequencies

fs_dict = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700}

label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
    
sec = 12
N = fs_dict['BVP']*sec  # one block : 10 sec
overlap = int(np.round(N * 0.02)) # overlapping length
overlap = overlap if overlap%2 ==0 else overlap+1

In [38]:
class SubjectData:

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'Resp': self.data['signal']['chest']['Resp']})
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results

In [39]:
def extract_ppg_data(e4_data_dict, labels, norm_type=None):
    # Dataframes for each sensor type
    df = pd.DataFrame(e4_data_dict['BVP'], columns=['BVP'])
    label_df = pd.DataFrame(labels, columns=['label'])
    

    # Adding indices for combination due to differing sampling frequencies
    df.index = [(1 / fs_dict['BVP']) * i for i in range(len(df))]
    label_df.index = [(1 / fs_dict['label']) * i for i in range(len(label_df))]

    # Change indices to datetime
    df.index = pd.to_datetime(df.index, unit='s')
    label_df.index = pd.to_datetime(label_df.index, unit='s')

    df = df.join(label_df, how='outer')
    
    df['label'] = df['label'].fillna(method='bfill')
    
    df.reset_index(drop=True, inplace=True)
    
    if norm_type == 'std':  # 시그널 자체를 normalization
        # std norm
        df['BVP'] = (df['BVP'] - df['BVP'].mean()) / df['BVP'].std()
    elif norm_type == 'minmax':
        # minmax norm
        df = (df - df.min()) / (df.max() - df.min())

    # Groupby
    df = df.dropna(axis=0) # nan인 행 제거
    
    return df


In [40]:
def seperate_data_by_label(df):
    
    grouped = df.groupby('label')
    baseline = grouped.get_group(1)
    stress = grouped.get_group(2)
    amusement = grouped.get_group(3)
    meditation = grouped.get_group(4)
    
    return grouped, baseline, stress, amusement, meditation

In [41]:
def get_samples(data, label, ma_usage):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []

    window_len = fs_dict['BVP'] * WINDOW_IN_SECONDS  # 64*60 , sliding window: 0.25 sec (60*0.25 = 15)   
    sliding_window_len = int(fs_dict['BVP'] * WINDOW_IN_SECONDS * 0.25)
    
    winNum = 0
    
    i = 0
    while sliding_window_len * i <= len(data) - window_len:
        
         # 한 윈도우에 해당하는 모든 윈도우 담기,
        w = data[sliding_window_len * i: (sliding_window_len * i) + window_len]  
        # Calculate stats for window
        wstats = get_window_stats_27_features(ppg_seg=w['BVP'].tolist(), window_length = window_len, label=label, ensemble = ENSEMBLE, ma_usage = ma_usage)
        winNum += 1
        
        if wstats == []:
            i += 1
            continue;
        # Seperating sample and label
        x = pd.DataFrame(wstats, index = [i])
    
        samples.append(x)
        i += 1

    return pd.concat(samples)

In [42]:
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_feats_4.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    df['label'] = (df['0'].astype(str) + df['1'].astype(str)).apply(lambda x: x.index('1'))  # 1인 부분의 인덱스 반환
    df.drop(['0', '1'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(savePath + merged_path)

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')

In [43]:
def make_patient_data(subject_id, ma_usage):
    global savePath
    global WINDOW_IN_SECONDS

    temp_ths = [1.0,2.0,1.8,1.5] #[1.1,2.2,2.0,1.9] #[1.1,2.2,1.8,1.9]
    clean_df = pd.read_csv('clean_signal_by_rate.csv',index_col=0)
    cycle = 15
    
    # Make subject data object for Sx
    subject = SubjectData(main_path=main_path, subject_number=subject_id)
    
    # Empatica E4 data
    e4_data_dict = subject.get_wrist_data()

    # norm type
    norm_type = 'std'

    df = extract_ppg_data(e4_data_dict, subject.labels, norm_type)
    df_BVP = df.BVP


    #여기서 signal preprocessing 
    bp_bvp = butter_bandpassfilter(df_BVP, 0.5, 10, fs_dict['BVP'], order=2) # 0.5, 5 -> 0.5,10
    
    if BP:     
        df['BVP'] = bp_bvp
        
    if FREQ:
        signal_one_percent = int(len(df_BVP) * 0.01)
        print(signal_one_percent)
        cutoff = get_cutoff(df_BVP[:signal_one_percent], fs_dict['BVP'])
        freq_signal = compute_and_reconstruction_dft(df_BVP, fs_dict['BVP'], sec, overlap, cutoff)
        df['BVP'] = freq_signal

    if TIME:
        fwd = moving_average(bp_bvp, size=3)
        bwd = moving_average(bp_bvp[::-1], size=3)
        bp_bvp = np.mean(np.vstack((fwd,bwd[::-1])), axis=0)
        df['BVP'] = bp_bvp
        
        signal_01_percent = int(len(df_BVP) * 0.001)
        print(signal_01_percent, int(clean_df.loc[subject_id]['index']))
        clean_signal = df_BVP[int(clean_df.loc[subject_id]['index']):int(clean_df.loc[subject_id]['index'])+signal_01_percent]
        ths = statistic_threshold(clean_signal, fs_dict['BVP'], temp_ths)
        len_before, len_after, time_signal_index = eliminate_noise_in_time(df['BVP'].to_numpy(), fs_dict['BVP'], ths, cycle)
    
        df = df.iloc[time_signal_index,:]
        df = df.reset_index(drop=True)
        #plt.figure(figsize=(40,20))
        #plt.plot(df['BVP'][:2000], color = 'b', linewidth=2.5)
    
    
    grouped, baseline, stress, amusement, meditation = seperate_data_by_label(df)   
    
    
    
    baseline_samples = get_samples(baseline, 1, ma_usage)
    print("bas: ",len(baseline_samples))
    stress_samples = get_samples(stress, 2, ma_usage)
    print("st: ",len(stress_samples))
    amusement_samples = get_samples(amusement, 0, ma_usage)
    print("Am: ",len(amusement_samples))
    meditation_samples = get_samples(meditation, 3, ma_usage)
    print("Me: ",len(meditation_samples))
    window_len = len(baseline_samples)+len(stress_samples)+len(amusement_samples)+len(meditation_samples)

    all_samples = pd.concat([baseline_samples, stress_samples, amusement_samples, meditation_samples])
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1) # get dummies로 원핫벡터로 라벨값 나타냄
    
    
    all_samples.to_csv(f'{savePath}{subject_feature_path}/S{subject_id}_feats_4.csv')

    # Does this save any space?
    subject = None
    
    return window_len


In [44]:
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_feats_4.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str) + df['3'].astype(str)).apply(lambda x: x.index('1'))  # 1인 부분의 인덱스 반환
    df.drop(['0', '1', '2', '3'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(savePath + merged_path)

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')


# +
total_window_len = 0
BP, FREQ, TIME, ENSEMBLE = False, False, False, False
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

feat_names = None
savePath = '27_features_ppg_test_4/LMM'

if not os.path.exists(savePath):
    os.makedirs(savePath)


for n in NOISE:
    if 'bp' in n.split('_'):
        BP = True
    if 'time' in n.split('_'):
        TIME = True
    if 'ens' in n.split('_'):
        ENSEMBLE = True
        


    subject_feature_path = '/subject_feature_' + n + str(WINDOW_IN_SECONDS)
    merged_path = '/data_merged_' + n + str(WINDOW_IN_SECONDS) +'.csv'
    
    if not os.path.exists(savePath + subject_feature_path):
        os.makedirs(savePath + subject_feature_path)
    
        
    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        window_len = make_patient_data(patient, BP)
        total_window_len += window_len

    combine_files(subject_ids)
    print('total_Window_len: ',total_window_len)
    print('Processing complete.', n)
    total_window_len = 0
# -

Processing data for S2...
389 1500
389056 338602


/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(me

bas:  32
st:  14
Am:  9
Me:  21
Processing data for S3...
415 16000
415552 368357


/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


bas:  29


...

/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


bas:  34
st:  18
Am:  9


/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Me:  21
Number of samples per class:
baseline: 508


KeyError: 3